# Web Scraping con Python

### Librerías

Vamos a necesitar Selenium, BeautifulSoup y Pandas

Instala Selenium con el siguiente código *conda install -c conda-forge selenium*

Instala BeautifulSoup con el código *conda install -c anaconda beautifulsoup4*

Ahora Carga Webdriver de Selenium

In [16]:
from selenium import webdriver
driver = webdriver.Chrome()

Carga BeautifulSoup de Beautiful Soup

In [17]:
from bs4 import BeautifulSoup

Y Pandas como PD

In [18]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver.get("https://vplaces.com")

try:
    elemento = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "mi_id_de_ejemplo"))
    )
    print(elemento.text)
except:
    print("No se encontró el elemento")

No se encontró el elemento


## Configuración

Arma 3 listas, una para metros cuadrados, otra para precio y otra para ubicacion

In [19]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

In [20]:
options = webdriver.ChromeOptions()
options.add_argument("--proxy-server=http://192.168.0.100:8080")
options.add_experimental_option("prefs", {
    "profile.default_content_setting_values.geolocation": 1
})

Y arma un driver.get para especificar la página de donde vamos a trabajar

In [21]:
driver.get("http://www.mineduc.cl/temas-educacion/formacion-continua/index.html")

## Extracción de Datos

Guarda el código HTML de la página en un objeto Contenido

In [12]:
contenido_html = driver.page_source

In [13]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(contenido_html, "html.parser")

Conviertelo a BeautifulSoup

In [22]:
contenido_html = driver.page_source

In [23]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(contenido_html, "html.parser")

Arma un for en el que vayas agregando a nuestras listas los datos necesarios con a.find, usa . append para ir agregando los datos a las listas

In [24]:
titulos = []
enlaces = []       

In [25]:
for elemento in soup.find_all("a"):
    titulo = elemento.get_text(strip=True)
    enlace = elemento.get("href", "")
    titulos.append(titulo)
    enlaces.append(enlace)

In [26]:
print("Primeros 5 títulos:", titulos[:5])
print("Primeros 5 enlaces:", enlaces[:5])

Primeros 5 títulos: ['MENU', 'Inicio', 'Ministerio►', 'Ministerio', 'Ministro de Educación']
Primeros 5 enlaces: ['#', 'https://www.mineduc.cl', '#', 'https://www.mineduc.cl/ministerio/', 'https://www.mineduc.cl/ministerio/ministro-de-educacion/']


In [27]:
import pandas as pd

df = pd.DataFrame({
    "Titulo": titulos,
    "Enlace": enlaces
})
df.head()

,Titulo,Enlace
0,MENU,#
1,Inicio,https://www.mineduc.cl
2,Ministerio►,#
3,Ministerio,https://www.mineduc.cl/ministerio/
4,Ministro de Educación,https://www.mineduc.cl/ministerio/ministro-de-...


In [28]:
df.to_csv("scraping_resultados.csv", index=False)
print("Datos guardados en 'scraping_resultados.csv'")

Datos guardados en 'scraping_resultados.csv'


Arma un for para limpiar precio

In [29]:
lista_precios = []

for elemento in soup.find_all("span", class_="precio"):
    precio_texto = elemento.get_text(strip=True)
    precio_valor = float(precio_texto.replace("$", "").replace(",", ""))
    lista_precios.append(precio_valor)

print("Precios encontrados:", lista_precios)

Precios encontrados: []


In [30]:
precios_filtrados = []

for precio in lista_precios:
    if precio < 50:  # Ajusta el valor según el criterio de filtrado
        precios_filtrados.append(precio)

print("Precios filtrados (< $50):", precios_filtrados)

Precios filtrados (< $50): []


Arma un for para limpiar ubicación

In [31]:
lista_ubicaciones = []

for elemento in soup.find_all("span", class_="ubicacion"):
    ubicacion_texto = elemento.get_text(strip=True)
    lista_ubicaciones.append(ubicacion_texto)

print("Ubicaciones encontradas:", lista_ubicaciones)

Ubicaciones encontradas: []


In [32]:
ubicaciones_filtradas = []

for ubicacion in lista_ubicaciones:
    if "Santiago" in ubicacion:
        ubicaciones_filtradas.append(ubicacion)

print("Ubicaciones filtradas (contienen 'Santiago'):", ubicaciones_filtradas)

Ubicaciones filtradas (contienen 'Santiago'): []


Arma un for para limpiar m2

In [33]:
n2 = ["   12.5 ", "  $15,00 ", "20,30 ", " $  7.90"]

n2_limpio = []

for elemento in n2:
    temp = elemento.strip()
    temp = temp.replace("$", "").replace(",", "")
    valor = float(temp)
    n2_limpio.append(valor)

print("n2 original:", n2)
print("n2 limpio:", n2_limpio)

n2 original: ['   12.5 ', '  $15,00 ', '20,30 ', ' $  7.90']
n2 limpio: [12.5, 1500.0, 2030.0, 7.9]


In [34]:
n2_filtrado = []

for valor in n2_limpio:
    if valor > 10:
        n2_filtrado.append(valor)

print("Valores mayores a 10:", n2_filtrado)

Valores mayores a 10: [12.5, 1500.0, 2030.0]


## Almacenamiento

Guarda tus datos en un dataframe

In [35]:
import pandas as pd

df_datos = pd.DataFrame({
    "Ubicaciones": lista_ubicaciones,
    "Precios": lista_precios
})
df_datos.head()

,Ubicaciones,Precios


In [36]:
df_datos.to_csv("datos_scraping.csv", index=False)
print("Datos guardados en 'datos_scraping.csv'")

Datos guardados en 'datos_scraping.csv'


Exporta tu Dataframe a un archivo CSV y abrelo en Excel

In [37]:

df_datos.to_csv("datos_scraping.csv", index=False)